## Image Classification using Bidirectional LSTM

In [0]:
import os
from keras.models import Model
from keras.layers import (Input,LSTM,GRU,Bidirectional,
                          GlobalMaxPooling1D,Lambda,Concatenate,Dense)
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
def get_mnist():
  print('Reading and transforming data...')
  df = pd.read_csv('train.csv')
  data = df.values
  np.random.shuffle(data)
  X = data[:,1:].reshape(-1,28,28)/255
  Y = data[:,0]
  return X,Y

In [22]:
X,Y = get_mnist()

Reading and transforming data...


In [0]:
# config
D = 28
M = 15


# input is an image of size 28x28
input_ = Input(shape=(D, D))

# up-down
rnn1 = Bidirectional(LSTM(M, return_sequences=True))
x1 = rnn1(input_) # output is N x D x 2M
x1 = GlobalMaxPooling1D()(x1) # output is N x 2M

# left-right
rnn2 = Bidirectional(LSTM(M, return_sequences=True))

# custom layer
permutor = Lambda(lambda t: K.permute_dimensions(t, pattern=(0, 2, 1)))

x2 = permutor(input_)
x2 = rnn2(x2) # output is N x D x 2M
x2 = GlobalMaxPooling1D()(x2) # output is N x 2M

# put them together
concatenator = Concatenate(axis=1)
x = concatenator([x1, x2]) # output is N x 4M

# final dense layer
output = Dense(10, activation='softmax')(x)

model = Model(inputs=input_, outputs=output)

# testing
# o = model.predict(X)
# print("o.shape:", o.shape)

# compile
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [0]:
# train
print('Training model...')
r = model.fit(X, Y, batch_size=32, epochs=10, validation_split=0.3)